In [1]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup
from tokenizers import ByteLevelBPETokenizer
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split

# [Previous imports and configurations remain the same]

# Set the visible GPU devices
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


/home/el/miniconda3/envs/nmt2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def calculate_perplexity(model, eval_dataloader, device):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for batch in tqdm(eval_dataloader, desc="Calculating perplexity"):
            batch = {k: v.to(device) for k, v in batch.items()}
            
            # Get number of tokens excluding padding
            num_tokens = batch['attention_mask'].sum().item()
            
            outputs = model(**batch)
            loss = outputs.loss
            
            # Accumulate loss and token count
            total_loss += loss.item() * num_tokens
            total_tokens += num_tokens
    
    # Calculate average negative log likelihood
    avg_nll = total_loss / total_tokens
    
    # Calculate perplexity
    perplexity = np.exp(avg_nll)
    
    return perplexity


In [3]:
tokenizer_path = '/DATA3/vaibhav/isign/PretrainingISL/helpers/custom_tokenizer'
model_path = '/DATA3/vaibhav/isign/PretrainingISL/helpers/custom_gpt2/best_model'


In [ ]:
trained_model = GPT2LMHeadModel.from_pretrained(model_path)  #config=decoder_config
print("Model loaded successfully")

In [ ]:
final_perplexity = calculate_perplexity(trained_model, val_dataloader, device)
print(f"Final validation perplexity: {final_perplexity:.4f}")

    